<a href="https://colab.research.google.com/github/NirantK/Hinglish/blob/cleanlab/CleanlabDistilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cleanlab

In [2]:
import pandas as pd
from utils import get_files_from_gdrive
url = "https://drive.google.com/file/d/1T6N_ba6-w2xLM6t4_EINGR7SGVQpCXd_/view?usp=sharing"

get_files_from_gdrive(url, "data/raw/distilBertOutput.csv")
DistilBert = pd.read_csv('data/raw/distilBertOutput.csv')
DistilBert

Downloading...
From: https://drive.google.com/uc?id=1T6N_ba6-w2xLM6t4_EINGR7SGVQpCXd_
To: /Users/meghanabhange/Documents/repos/Hinglish/data/raw/distilBertOutput.csv
100%|██████████| 399k/399k [00:00<00:00, 937kB/s]


,Uid,Sentiment,clean_text
0,20803,neutral,_ Agr kse ko itni importantce chaeay ni tou ...
1,20187,neutral,logon ko alloo pyaz tomator me toh allah pak k...
2,20953,neutral,Wafadaar bane rahane ka nayab tarika hai lamb...
3,13777,negative,Chup bhosdike . He has been exonerated of al...
4,20990,positive,à¨¦à¨¾à¨¤à¨¾ à¨¸à:copyright:‹ à¨¸à¨¾à¨²à¨¾à¨...
...,...,...,...
2994,7026,positive,Pradhanmantri ji aap ko dusri Baar Bharat ke ...
2995,10425,positive,_ yogesh22 Haha Jai Ho Bhul Gye Haha Ham Fa...
2996,14162,positive,ðŸ:trade_mark:ðŸ:trade_mark:ðŸ:trade_mark: ...
2997,15860,neutral,_ Khan241 Fahad Bhaiiiii give us Bilal Abbas...


In [3]:
vals = {'neutral':0, 
        'negative':1,
        'positive':2}
DistilBert['labels'] = DistilBert.Sentiment.map(vals)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(DistilBert['clean_text'])
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

tfidf_transformer = TfidfTransformer()
sents = tfidf_transformer.fit_transform(X_train_counts)

In [5]:
DistilBert

,Uid,Sentiment,clean_text,labels
0,20803,neutral,_ Agr kse ko itni importantce chaeay ni tou ...,0
1,20187,neutral,logon ko alloo pyaz tomator me toh allah pak k...,0
2,20953,neutral,Wafadaar bane rahane ka nayab tarika hai lamb...,0
3,13777,negative,Chup bhosdike . He has been exonerated of al...,1
4,20990,positive,à¨¦à¨¾à¨¤à¨¾ à¨¸à:copyright:‹ à¨¸à¨¾à¨²à¨¾à¨...,2
...,...,...,...,...
2994,7026,positive,Pradhanmantri ji aap ko dusri Baar Bharat ke ...,2
2995,10425,positive,_ yogesh22 Haha Jai Ho Bhul Gye Haha Ham Fa...,2
2996,14162,positive,ðŸ:trade_mark:ðŸ:trade_mark:ðŸ:trade_mark: ...,2
2997,15860,neutral,_ Khan241 Fahad Bhaiiiii give us Bilal Abbas...,0


In [6]:
import cleanlab

In [7]:
py_train = cleanlab.util.value_counts(DistilBert['labels']) / float(len(DistilBert['labels']))
py_train

0    0.379126
2    0.316439
1    0.304435
Name: labels, dtype: float64

In [8]:
prune_method = 'prune_by_noise_rate'

In [9]:
import numpy as np
np.array(list(DistilBert['labels']))

array([0, 0, 0, ..., 2, 0, 0])

In [10]:
from sklearn.linear_model import LogisticRegression as LogReg

(jc, psx) = cleanlab.latent_estimation.estimate_confident_joint_and_cv_pred_proba(
    sents,
    np.array(list(DistilBert["labels"])),
    clf=LogReg(multi_class="auto", solver="lbfgs", max_iter=10000, verbose=True),
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished


In [11]:
est_py, est_nm, est_inv = cleanlab.latent_estimation.estimate_latent(
    jc, np.array(list(DistilBert["labels"]))
)

In [12]:
ordered = cleanlab.pruning.get_noise_indices(
    np.array(list(DistilBert["labels"])),
    psx,
    #    est_inv,
    prune_method=prune_method,
    frac_noise=0.37,
    sorted_index_method="normalized_margin",
)
print("Number of estimated errors in test set:", len(ordered))

Number of estimated errors in test set: 263


In [13]:
len(ordered)

263

In [14]:
vals = {0: "negative", 1: "neutral", 2: "positive"}
errors = []
for idx in ordered[:100]:
    errors.append(
        f"Estimated Error in {idx}\npredicted {DistilBert['Sentiment'][idx]}\n sentence {DistilBert['clean_text'][idx]}\n -----"
    )

In [15]:
with open('data/cleanlab/errorsDistilBert.txt', 'w') as f: 
    for item in errors:
        f.write("%s\n" % item)